In [7]:
# Get tensorboard running in the background
LOG_DIR = '/tmp/log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

# Download and unzip ngrok
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

# Launch ngrok background process
get_ipython().system_raw('./ngrok http 6006 &')

# Retrive public url
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


Redirecting output to ‘wget-log’.
Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://1c743d52.ngrok.io


In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will proot for authorization
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

# NAME = "Cats-vs-dog-cnn-64x2-{}".format(int(time.time()))


X = pickle.load(open("/content/drive/My Drive/Colab Notebooks/cnn_own_data/X.pickle", "rb"))
y = pickle.load(open("/content/drive/My Drive/Colab Notebooks/cnn_own_data/y.pickle", "rb"))

X = X/255.0

dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
  for layer_size in layer_sizes:
    for conv_layer in conv_layers:
      NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
      print(NAME)
      tensorboard = TensorBoard(log_dir='/tmp/log/{}'.format(NAME)) # TensorBoard(log_dir='logs/{}'.format(NAME))
      
      model = Sequential()
      
      model.add(Conv2D(layer_size, (3,3), input_shape=X.shape[1:]))
      model.add(Activation("relu"))
      model.add(MaxPooling2D(pool_size=(2,2)))
      
      for l in range(conv_layer-1):
        model.add(Conv2D(layer_size, (3,3)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2)))
      
      model.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
      for l in range(dense_layer):
        model.add(Dense(layer_size))
        model.add(Activation('relu'))

      model.add(Dense(1))
      model.add(Activation("sigmoid"))

      model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

      model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

'''
Best versions of the conv network (used tensorboard to analyze data)
3 conv, 64 nodes per layer, 0 dense
3 conv, 128 nodes per layer, 0 dense
3 conv, 32 nodes per layer, 0 dense
3 conv, 32 nodes per layer, 2 dense
3 conv, 32 nodes per layer, 1 dense
2 conv, 32 nodes per layer, 0 dense
2 conv, 64 nodes per layer, 0 dense
3 conv, 128 nodes per layer, 1 dense
2 conv, 128 nodes per layer, 0 dense
2 conv, 32 nodes per layer, 1 dense
'''